# Introduction to pyMOR

## What is pyMOR?

* #### A software library for building model order reduction applications with *Python*
* #### Free and open source

## How did pyMOR start?

* #### Project started in October 2012
* #### Founding members were based at the University of Münster, Germany
* #### Started off as a library for applying the reduced-basis method
* #### Original motivation: Decouple MOR algorithms from large-scale models
    * #### Proprietary code for battery simulation models could not be shared with MOR experts
    * #### Can we still do model reduction without access to the model?

<table width=70%>
    <tr>
        <td><img src="figs/renefritze.jpg"></td>
        <td><img src="figs/sdrave.jpg"></td>
        <td><img src="figs/ftschindler.jpg"></td>
    </tr>
    <tr>
        <td>René Fritze</td>
        <td>Stephan Rave</td>
        <td>Felix Schindler</td>
    </tr>
</table>
<br>




## Where is pyMOR now?

* #### Over 40 people have contributed, with 5 designated main developers
* #### Over 25k lines of code
* #### Annual pyMOR school events with lectures and project support
* #### Linear algebra and MOR algorithms from various research communities:
    * #### Randomized linear algebra (SVD, eigenvalue solvers, ...)
    * #### Balanced Truncation
    * #### IRKA
    * #### AAA/Loewner
    * #### Reduced basis method (RBM)
    * #### POD
    * #### ...

<table width=70%>
    <tr>
        <td><img src="figs/pmli.jpg"></td>
        <td><img src="figs/HenKlei.jpg"></td>
        <td><img src="figs/sdrave.jpg"></td>
        <td><img src="figs/ftschindler.jpg"></td>
        <td><img src="figs/lbalicki.jpg"></td>
    </tr>
    <tr>
        <td>Petar Mlinarić</td>
        <td>Hendrik Kleikamp</td>
        <td>Stephan Rave</td>
        <td>Felix Schindler</td>
        <td>Linus Balicki</td>
    </tr>
</table>
<br>
<br>
<br>

## Imports and Settings

In [ ]:
import numpy as np
import scipy.linalg as spla
import matplotlib.pyplot as plt
from pymor.core.logger import set_log_levels

plt.rcParams['axes.grid'] = True
set_log_levels({
    'pymor.algorithms.gram_schmidt.gram_schmidt': 'WARNING',
    'pymor.reductors.basic.LTIPGReductor': 'WARNING',
})

## pyMOR Structure

<img src="figs/pyMOR-structure.jpg">

## LTI Models in pyMOR

#### How do we create a basic LTI model?

$$
\begin{align*}
\dot{\mathbf{x}}(t) &= \mathbf{A} \mathbf{x}(t) + \mathbf{B} u(t) \\ 
y(t) &= \mathbf{C} \mathbf{x}(t)
\end{align*}
$$

In [ ]:
from pymor.models.iosys import LTIModel

A = np.random.rand(10,10)
B = np.random.rand(10,1)
C = np.random.rand(1,10)

lti = LTIModel.from_matrices(A,B,C)

In [ ]:
lti

In [ ]:
lti.A

#### What are these NumpyMatrixOperators?

* #### pyMOR supports various Operators which can handle many different things
    * #### "Black-box" evaluations coupled to an external PDE solver
    * #### MPI distributed computations
    * #### Structured matrices (banded, block, Hankel, ...)
    * #### Nonlinear operators (e.g. quadratic $\mathbf{N} (\mathbf{x}(t) \otimes \mathbf{x}(t)$)
    * #### Simply a numpy matrix &#8594; NumpyMatrixOperator
* #### Operators allow for decoupling the MOR algorithms from the linear algebra backend of the large-scale model

In [ ]:
lti.A.matrix

In [ ]:
print(lti)

## Rail Model

In [ ]:
from pymor.models.iosys import LTIModel

rail_fom = LTIModel.from_abcde_files('data/rail/rail_5177_c60')

In [ ]:
rail_fom

In [ ]:
rail_fom.A

In [ ]:
rail_fom.A.matrix

In [ ]:
print(rail_fom)

In [ ]:
w = (1e-7, 1e3)
_ = rail_fom.transfer_function.mag_plot(w)

## Hankel Singular Values

In [ ]:
rail_hsv = rail_fom.hsv()

# %% slideshow={"slide_type": "fragment"}
fig, ax = plt.subplots()
_ = ax.semilogy(rail_hsv, '.')
_ = ax.set_title('Hankel singular values')

## Balanced Truncation

In [ ]:
from pymor.reductors.bt import BTReductor

rail_bt = BTReductor(rail_fom)

rail_rom_bt = rail_bt.reduce(20)

rail_rom_bt

In [ ]:
rail_rom_bt.A.matrix

## BT Poles

In [ ]:
fig, ax = plt.subplots()
poles = rail_rom_bt.poles()
_ = ax.plot(poles.real, poles.imag, '.')
_ = ax.set_title('BT poles')

In [ ]:
poles.real.max()

## FOM and BT Magnitude Plots

In [ ]:
w = (1e-6, 1e3)
_ = rail_fom.transfer_function.mag_plot(w)
_ = rail_rom_bt.transfer_function.mag_plot(w)

## BT Error System

In [ ]:
rail_err_bt = rail_fom - rail_rom_bt

In [ ]:
rail_err_bt

In [ ]:
_ = rail_err_bt.transfer_function.mag_plot(w)

## $\mathcal{H}_2$ Relative Error

In [ ]:
rail_err_bt.h2_norm() / rail_fom.h2_norm()

In [ ]:
_ = plt.semilogy(rail_bt.error_bounds(), '.-')

## IRKA

In [ ]:
from pymor.reductors.h2 import IRKAReductor

rail_irka = IRKAReductor(rail_fom)

In [ ]:
rail_rom_irka = rail_irka.reduce(20, conv_crit='h2', tol=1e-3, num_prev=10)

In [ ]:
rail_rom_irka

## IRKA Convergence

In [ ]:
_ = plt.semilogy(rail_irka.conv_crit, '.-')

## IRKA Poles

In [ ]:
fig, ax = plt.subplots()
poles = rail_rom_irka.poles()
_ = ax.plot(poles.real, poles.imag, '.')
_ = ax.set_title('IRKA poles')


In [ ]:
poles.real.max()

## FOM and IRKA Magnitude Plots

In [ ]:
w = (1e-6, 1e3)
_ = rail_fom.transfer_function.mag_plot(w)
_ = rail_rom_irka.transfer_function.mag_plot(w)

## IRKA Error System

In [ ]:
rail_err_irka = rail_fom - rail_rom_irka

In [ ]:
fig, ax = plt.subplots()
_ = rail_err_bt.transfer_function.mag_plot(w, ax=ax, label='BT')
_ = rail_err_irka.transfer_function.mag_plot(w, ax=ax, label='IRKA')
_ = ax.legend()

## $\mathcal{H}_2$ Relative Error

In [ ]:
rail_err_irka.h2_norm() / rail_fom.h2_norm()

## Transfer Function

#### Heat equation over a semi-infinite rod from \[Beattie/Gugercin '12\].

$$
\begin{align*}
  H(s) & = e^{-\sqrt{s}} \\
  H'(s) & = -\frac{e^{-\sqrt{s}}}{2 \sqrt{s}}
\end{align*}
$$

In [ ]:
from pymor.models.transfer_function import TransferFunction

def H(s):
    return np.array([[np.exp(-np.sqrt(s))]])

def dH(s):
    return np.array([[-np.exp(-np.sqrt(s)) / (2 * np.sqrt(s))]])

tf = TransferFunction(
    1,
    1,
    H,
    dH,
)

In [ ]:
tf

## Bode Plot

In [ ]:
w_tf = (1e-7, 1e4)
fig, ax = plt.subplots(2, 1, figsize=(6, 6), sharex=True, squeeze=False, constrained_layout=True)
_ = tf.bode_plot(w_tf, ax=ax)


## TF-IRKA

In [ ]:
from pymor.reductors.h2 import TFIRKAReductor

tf_irka = TFIRKAReductor(tf)

In [ ]:
tf_rom = tf_irka.reduce(20)

In [ ]:
tf_rom

## TF-IRKA Poles

In [ ]:
fig, ax = plt.subplots()
poles = tf_rom.poles()
_ = ax.plot(poles.real, poles.imag, '.')
_ = ax.set_title('IRKA poles')

In [ ]:
poles.real.max()

## Bode Plots

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(6, 6), sharex=True, squeeze=False, constrained_layout=True)
_ = tf.bode_plot(w_tf, ax=ax)
_ = tf_rom.transfer_function.bode_plot(w_tf, ax=ax)

## Error System

In [ ]:
tf_err = tf - tf_rom

In [ ]:
tf_err

In [ ]:
_ = tf_err.mag_plot(w_tf)

## Loewner

In [ ]:
from pymor.reductors.loewner import LoewnerReductor

sampling_values = 1j * np.logspace(-4, 2, 100)
sampling_values = np.concatenate((sampling_values, -sampling_values))
loewner = LoewnerReductor(sampling_values, tf)

In [ ]:
loewner_rom = loewner.reduce(r=20)

In [ ]:
loewner_rom

In [ ]:
L, _, _, _ = loewner.loewner_quadruple()
_, S, _ = spla.svd(L)

plt.semilogy(S/S[0])

## Bode Plots

In [ ]:
fig, ax = plt.subplots(2, 1, squeeze=False, constrained_layout=True)
_ = tf.bode_plot(w_tf, ax=ax)
_ = tf_rom.transfer_function.bode_plot(w_tf, ax=ax)
_ = loewner_rom.transfer_function.bode_plot(w_tf, ax=ax)

## Error System

In [ ]:
loewner_err = tf - loewner_rom

In [ ]:
fig, ax = plt.subplots()
_ = tf_err.mag_plot(w_tf, ax=ax, label='TF-IRKA')
_ = loewner_err.mag_plot(w_tf, ax=ax, label='Loewner')
_ = ax.legend()

## Parametric LTI Models

#### Cookie model (thermal block) example from [MOR Wiki](https://morwiki.mpi-magdeburg.mpg.de/morwiki/index.php/Thermal_Block).


In [ ]:
import scipy.io as spio

mat = spio.loadmat('data/cookie/ABCE.mat')

In [ ]:
mat.keys()

In [ ]:
A0 = mat['A0']
A1 = 0.2 * mat['A1'] + 0.4 * mat['A2'] + 0.6 * mat['A3'] + 0.8 * mat['A4']
B = mat['B']
C = mat['C']
E = mat['E']

In [ ]:
A0

In [ ]:
from pymor.operators.numpy import NumpyMatrixOperator

A0op = NumpyMatrixOperator(A0)
A1op = NumpyMatrixOperator(A1)
Bop = NumpyMatrixOperator(B)
Cop = NumpyMatrixOperator(C)
Eop = NumpyMatrixOperator(E)

In [ ]:
A0op

In [ ]:
from pymor.parameters.functionals import ProjectionParameterFunctional

Aop = A0op + ProjectionParameterFunctional('p') * A1op

In [ ]:
Aop

In [ ]:
cookie_fom = LTIModel(Aop, Bop, Cop, E=Eop)

In [ ]:
cookie_fom

In [ ]:
cookie_fom.parameters

## Magnitude Plot

In [ ]:
num_w = 10
num_p = 10
ws = np.logspace(-4, 4, num_w)
ps = np.logspace(-6, 2, num_p)
Hwp = np.empty((num_p, num_w))
for i in range(num_p):
    Hwp[i] = spla.norm(cookie_fom.transfer_function.freq_resp(ws, mu=ps[i]), axis=(1, 2))

In [ ]:
from matplotlib.colors import LogNorm

lognorm = LogNorm(vmin=9e-11, vmax=2)

fig, ax = plt.subplots()
out = ax.pcolormesh(ws, ps, Hwp, shading='gouraud', norm=lognorm)
ax.set(
    xscale='log',
    yscale='log',
    xlabel=r'Frequency $\omega$ (rad/s)',
    ylabel='Parameter $p$',
    title=r'$\Vert H(i \omega, p) \Vert$',
)
ax.grid(False)
_ = fig.colorbar(out)


## Interpolation

In [ ]:
from pymor.algorithms.gram_schmidt import gram_schmidt
from pymor.reductors.interpolation import LTIBHIReductor

s_samples = np.logspace(-1, 1, 5)
s_samples = np.concatenate((1j * s_samples, -1j * s_samples))
p_samples = np.logspace(-3, -1, 5)
V = cookie_fom.A.source.empty()
W = cookie_fom.A.source.empty()
for p in p_samples:
    interp = LTIBHIReductor(cookie_fom, mu=p)
    interp.reduce(s_samples, np.ones((len(s_samples), 1)), np.ones((len(s_samples), 4)))
    V.append(interp.V)
    W.append(interp.W)

_ = gram_schmidt(V, copy=False)
_ = gram_schmidt(W, copy=False)


In [ ]:
V

In [ ]:
from pymor.reductors.basic import LTIPGReductor

pg = LTIPGReductor(cookie_fom, W, V)
cookie_rom = pg.reduce()

In [ ]:
cookie_rom

## Error System

In [ ]:
cookie_err = cookie_fom - cookie_rom

In [ ]:
Hwp_err = np.empty((num_p, num_w))
for i in range(num_p):
    Hwp_err[i] = spla.norm(cookie_err.transfer_function.freq_resp(ws, mu=ps[i]), axis=(1, 2))


In [ ]:
fig, ax = plt.subplots()
out = ax.pcolormesh(ws, ps, Hwp_err, shading='gouraud', norm=lognorm)
ax.set(
    xscale='log',
    yscale='log',
    xlabel=r'Frequency $\omega$ (rad/s)',
    ylabel='Parameter $p$',
    title=r'$\Vert H(i \omega, p) - H_r(i \omega, p) \Vert$',
)
ax.grid(False)
_ = fig.colorbar(out)


## ROM Poles

In [ ]:
for p in ps:
    poles = cookie_rom.poles(mu=p)
    print(poles.real.max())

## Galerkin Projection

In [ ]:
from pymor.algorithms.pod import pod

VW = V.copy()
VW.append(W)
VW, svals = pod(VW)

In [ ]:
VW

In [ ]:
galerkin = LTIPGReductor(cookie_fom, VW, VW)
cookie_rom_g = galerkin.reduce()

## Error System 2

In [ ]:
cookie_err2 = cookie_fom - cookie_rom_g

In [ ]:
Hwp_err2 = np.empty((num_p, num_w))
for i in range(num_p):
    Hwp_err2[i] = spla.norm(cookie_err2.transfer_function.freq_resp(ws, mu=ps[i]), axis=(1, 2))


In [ ]:
fig, ax = plt.subplots()
out = ax.pcolormesh(ws, ps, Hwp_err2, shading='gouraud', norm=lognorm)
ax.set(
    xscale='log',
    yscale='log',
    xlabel=r'Frequency $\omega$ (rad/s)',
    ylabel='Parameter $p$',
    title=r'$\Vert H(i \omega, p) - H_r(i \omega, p) \Vert$',
)
ax.grid(False)
_ = fig.colorbar(out)


## ROM Poles 2

In [ ]:
for p in ps:
    poles = cookie_rom_g.poles(mu=p)
    print(poles.real.max())

## Concluding Remarks

### Further MOR Methods

- #### nonlinear systems
- #### DEIM
- #### second-order systems
- #### time-delay systems
- #### modal truncation
- #### data-driven methods (p-AAA, Loewner, neural network based ROMs, ...)


## Using pyMOR

- #### installation: https://github.com/pymor/pymor#readme
- #### documentation: https://docs.pymor.org
- #### GitHub issues: https://github.com/pymor/pymor/issues
- #### GitHub discussions: https://github.com/pymor/pymor/discussions
- #### pyMOR community meetings: https://github.com/pymor/pymor/discussions/categories/announcements
- #### pyMOR School: https://school.pymor.org


## Contributing to pyMOR

- #### developer documentation: https://docs.pymor.org/latest/developer_docs.html
- #### get attribution via `AUTHORS.md`
- #### become contributor with push access to feature branches
- #### become main developer with full control over the project
